In [58]:
import numpy as np 
import pandas as pd 
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
from scipy import stats
from scipy.stats import norm, skew #for some statistics
import math
pd.set_option('display.max_rows', 3000)
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4
import math
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.preprocessing import RobustScaler

In [59]:
train = pd.read_csv("~/Downloads/Kaggle_Workflow/train.csv")
test = pd.read_csv("~/Downloads/Kaggle_Workflow/test.csv")

In [60]:
#Deleting outliers
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<200000)].index)

In [61]:
#Save the 'Id' column
train_ID = train['Id']
test_ID = test['Id']
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

In [62]:
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
train["SalePrice"] = np.log1p(train["SalePrice"])

In [63]:
ntrain = train.shape[0]
ntest = test.shape[0] #use this for splitting up all_data later
y_train = train.SalePrice.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
all_data.shape

(2917, 79)

In [64]:
#all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data = all_data.drop(labels = 'PoolQC', axis = 1)
#all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
# all_data = all_data.drop(labels = 'MiscFeature', axis = 1)
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")
#Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
#all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
#    lambda x: x.fillna(x.median()))
all_data["LotFrontage"] = all_data["LotFrontage"].fillna(0)
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')
for col in ('GarageYrBlt','GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data = all_data.drop(['Utilities'], axis=1)
all_data["Functional"] = all_data["Functional"].fillna("Typ")
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)
all_data['OverallCond'] = all_data['OverallCond'].astype(str)
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']
all_data['GrLivArea2'] = (all_data['GrLivArea']**(1/2))
all_data = all_data.drop('GrLivArea', axis = 1)
all_data['HalfBath2'] = all_data['HalfBath']
all_data.loc[all_data['HalfBath'] >= 1,'HalfBath2'] = .5
all_data['Bathrooms'] = all_data['FullBath'] + all_data['HalfBath2']
all_data = all_data.drop('FullBath', axis = 1)
all_data = all_data.drop('HalfBath', axis = 1)
all_data = all_data.drop('HalfBath2', axis = 1)
all_data['LotArea2'] = np.log(all_data['LotArea']+1)
all_data = all_data.drop('LotArea', axis = 1)
all_data['LotFrontage'] = all_data['LotFrontage'].astype(int)
all_data['LotFrontage2'] = (all_data['LotFrontage']**(1/2))
all_data = all_data.drop('LotFrontage', axis = 1)
all_data['ShedSF'] = 0
all_data.loc[all_data['MiscFeature'] == 'Shed', 'ShedSF'] = all_data['MiscVal']
all_data = all_data.drop('MiscFeature', axis = 1)
all_data = all_data.drop('MiscVal', axis = 1)
all_data = all_data.drop('TotalBsmtSF', axis = 1)
all_data = all_data.drop('LowQualFinSF', axis = 1)
# #3SsnPorch
all_data = all_data.drop('3SsnPorch', axis = 1)
# #Alley
# #93% of the data is NA
all_data = all_data.drop('Alley', axis = 1)
# #BldgType
# #Very unbalanced
all_data = all_data.drop('BldgType', axis = 1)
# #BsmtFinType1 and BsmtFinType2
# all_data['testBasementFinType'] = all_data['BsmtFinType1'].astype(str).str.cat(
#     all_data['BsmtFinType2'].astype(str), sep=' - ')
# all_data = all_data.drop('BsmtFinType1', axis = 1)
# all_data = all_data.drop('BsmtFinType2', axis = 1)
# #Condition2
# #99% of the data is Norm
all_data = all_data.drop('Condition2', axis = 1)
#Fence
all_data['HasFence'] = 0
all_data.loc[all_data['Fence'] != 'NA','HasFence'] = 1
all_data = all_data.drop('Fence', axis = 1)
#Garage
all_data = all_data.drop('GarageQual', axis = 1)

In [65]:
from sklearn.preprocessing import LabelEncoder
cols = ('GarageCond','FireplaceQu', 'BsmtQual', 'BsmtCond', 
        'ExterQual', 'ExterCond','HeatingQC', #'PoolQC', 
        'KitchenQual', 'Functional', 'BsmtFinType1', 'BsmtFinType2',
        'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 
        'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape))

Shape all_data: (2917, 69)


In [66]:
# numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
# # Check the skew of all numerical features
# skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
# print("\nSkew in numerical features: \n")
# skewness = pd.DataFrame({'Skew' :skewed_feats})
# skewness.head(10)

In [67]:
# skewness = skewness[abs(skewness) > 0.75]
# print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

# from scipy.special import boxcox1p
# skewed_features = skewness.index
# lam = 0.15
# for feat in skewed_features:
#    #all_data[feat] += 1
#    all_data[feat] = boxcox1p(all_data[feat], lam)
   
# all_data[skewed_features] = np.log1p(all_data[skewed_features])

In [68]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(2917, 195)


In [69]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [70]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [71]:
#best score is without removing logs/skewness (alpha = .0005, score = .1094)
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.cross_validation import cross_val_score
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
score = rmsle_cv(lasso)
print("lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
lasso.fit(train.values, y_train)

lasso score: 0.1094 (0.0061)



Pipeline(steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('lasso', Lasso(alpha=0.0005, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=1,
   selection='cyclic', tol=0.0001, warm_start=False))])

In [72]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0006, l1_ratio=.8, random_state=3))
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ElasticNet score: 0.1094 (0.0061)



In [73]:
#best score is without removing logs/skewness (alpha = 11, score = .1157)
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import linear_model

ridge = make_pipeline(RobustScaler(), linear_model.Ridge(alpha = 11, random_state=1))
score = rmsle_cv(ridge)
print("ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

#ridge = linear_model.Ridge(alpha = 5)
#ridge_scores = cross_val_score(ridge, housing_model, housing_target_train2, scoring='neg_mean_squared_error', cv = 5)
#ridge_scores = np.sqrt(-ridge_scores)
#ridge_predictions = cross_val_predict(ridge, housing_model, housing_target_train2, cv = 5)
#ridge.fit(housing_model, housing_target_train2)

ridge score: 0.1114 (0.0059)



In [50]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
parameters = {'kernel':['rbf'], 'C':[10,50,100], 'gamma':[1e-7, 1e-3],'epsilon':[0.00001,0.2]}
svc1 = svm.SVR()
clf1 = GridSearchCV(svc1, parameters, verbose = 1)
clf1.fit(train.values, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:   15.2s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [10, 50, 100], 'gamma': [1e-07, 0.001], 'kernel': ['rbf'], 'epsilon': [1e-05, 0.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [51]:
clf1.best_params_

{'C': 50, 'epsilon': 1e-05, 'gamma': 0.001, 'kernel': 'rbf'}

In [55]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.cross_validation import cross_val_score

clf = SVR(kernel = 'rbf', C= 50, gamma = .001, epsilon = 0.00001)
svm_cross_scores = cross_val_score(clf, train.values, y_train, cv=5, scoring ='neg_mean_squared_error')
final_crossval = np.sqrt(-svm_cross_scores)
final_crossval.mean()

0.12226348434957193

In [57]:
clf.fit(train.values, y_train)
svr_pred2 = clf.predict(test.values)

In [ ]:
svr_pred2

In [22]:
lasso20_pred = np.expm1(lasso.predict(test.values))
Lasso_submission20 = pd.DataFrame({'Id': test_ID, 'SalePrice': lasso20_pred})
Lasso_submission20.to_csv('Lasso_submission20.csv', index = False)

In [108]:
# #Getting best alpha for lasso
# from sklearn.model_selection import cross_val_score, cross_val_predict
# from sklearn import linear_model

# def rmse_cv(model):
#     rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = 5))
#     return(rmse)

# alphas = [0.0005, 0.001]
# cv_lasso = [rmse_cv(linear_model.Lasso(alpha = alpha)).mean() for alpha in alphas]

# cv_lasso = pd.Series(cv_lasso, index = alphas)
# cv_lasso.plot(title = "Validation")
# plt.xlabel("alpha")
# plt.ylabel("rmse")
# #alpha = 63 is the best option
# cv_lasso

In [53]:
#Bayesian Optimization
#Parameter tuning using Bayesian Optimization
#from bayes_opt import BayesianOptimization
#from sklearn.cross_validation import cross_val_score
#from sklearn import ensemble
#def gbcv(n_estimators, learning_rate, max_depth, min_samples_leaf, min_samples_split, subsample):
#    return cross_val_score(
#        ensemble.GradientBoostingRegressor(
#        n_estimators = int(n_estimators),
#        learning_rate = float(learning_rate),
#        max_depth = int(max_depth),
#        min_samples_leaf = int(min_samples_leaf),
#        min_samples_split = int(min_samples_split),
#        subsample = float(subsample),
#        random_state = 42),
#        train, y_train, 'neg_mean_squared_error', cv = 5
#    ).mean()
#
#gbBo = BayesianOptimization(gbcv,
#                            {'n_estimators' : (1000, 10000),
#                             'learning_rate' : (.01, .1),
#                             'max_depth' : (1,10),
#                             'min_samples_leaf' : (1,20),
#                             'min_samples_split' : (5,25),
#                             'subsample' : (1),
#                            }
#                            )
#
#gbBo.maximize(n_iter = 20)

In [54]:
#gbBo.res['max']

In [31]:
GBoost1 = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.0465,
                                   max_depth=2, max_features='sqrt',
                                   min_samples_leaf=3, min_samples_split=18, 
                                   loss='huber', random_state = 5)
#GBoost1 = GradientBoostingRegressor(n_estimators=7000, learning_rate=0.005,
#                                   max_depth=20, max_features='sqrt',
#                                   min_samples_leaf=5, min_samples_split=30, 
#                                   loss='huber', random_state = 5)

In [32]:
score = rmsle_cv(GBoost1)
print("Gradient Boosting1 score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Gradient Boosting1 score: 0.1099 (0.0086)



In [33]:
GBoost1.fit(train.values, y_train)
GB_pred = np.expm1(GBoost1.predict(test.values))

In [ ]:
GBSubmission = pd.DataFrame({ 'Id': test_ID,
                            'SalePrice': y_pred })
GBSubmission.to_csv("GBSubmission_train.csv", index=False)

In [ ]:
gb_feature_importance = GBoost1.feature_importances_
gb_feature_importance = 100.0 * (gb_feature_importance / gb_feature_importance.max())
#pyplot.bar(range(len(GBoost1.feature_importances_)), GBoost1.feature_importances_)
#pyplot.show()
feat_imp = pd.Series(GBoost1.feature_importances_, train.columns).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

In [ ]:
ENet.fit(train.values, y_train)
ENet_pred = np.expm1(ENet.predict(test.values))

In [ ]:
ENetSubmission = pd.DataFrame({ 'Id': test_ID,
                            'SalePrice': y_pred })
ENetSubmission.to_csv("ENetSubmission_test.csv", index=False)

In [ ]:



#clf = SVR(kernel = 'rbf', C= 10, gamma = .001, epsilon = 0.00005)
#svm_cross_scores = cross_val_score(clf, housing_model, housing_target_train2, cv=5, scoring ='neg_mean_squared_error')
#final_crossval = np.sqrt(-svm_cross_scores)
#final_crossval.mean()

In [562]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.cross_validation import cross_val_score
SVR = make_pipeline(SVR(kernel = 'rbf', C = 100, gamma = 0.001, epsilon = 0.0001))
score = rmsle_cv(SVR)
print("SVR score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

SVR score: 0.3958 (0.0167)



In [ ]:
SVR.fit(train.values, y_train)
svr_pred = np.expm1(SVR.predict(test.values))

In [ ]:
SVRSubmission = pd.DataFrame({ 'Id': test_ID,
                            'SalePrice': y_pred })
SVRSubmission.to_csv("SVRSubmission_test.csv", index=False)

In [ ]:
RobustScaler(train)

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
parameters = {'kernel':['rbf'], 'C':[100,500,1000], 'gamma':[1e-3, 1e-1],'epsilon':[0.00001,0.0001]}
svc1 = svm.SVR()
clf1 = GridSearchCV(svc1, parameters, verbose = 1)
clf1.fit(RobustScaler(train.values), y_train)

In [ ]:
clf1.best_params_

In [ ]:
lasso.fit(train.values, y_train)
lasso_pred = np.expm1(lasso.predict(test.values))

In [ ]:
lassoSubmission = pd.DataFrame({ 'Id': test_ID,
                            'SalePrice': y_pred })
lassoSubmission.to_csv("lassoSubmission_test.csv", index=False)

In [ ]:
#Getting best alpha for ridge
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import linear_model

def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

alphas = [.1,.5,1,2]
cv_ridge = [rmse_cv(linear_model.Ridge(alpha = alpha)).mean() for alpha in alphas]

cv_ridge = pd.Series(cv_ridge, index = alphas)
cv_ridge.plot(title = "Validation")
plt.xlabel("alpha")
plt.ylabel("rmse")
#alpha = 5 is the best option
cv_ridge

In [ ]:
y_pred = lasso_pred*(1/4) + svr_pred*(1/4) + GB_pred*(1/4) + ENet_pred*(1/4)

In [ ]:
score = rmsle_cv(y_pred)
print("ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
#y_pred.fit(train.values, y_train)
#avg_pred = np.expm1(y_pred.predict(test.values))

In [ ]:
avgSubmission = pd.DataFrame({ 'Id': test_ID,
                            'SalePrice': y_pred })
avgSubmission.to_csv("Submission_avg.csv", index=False)